## "감귤없이잘살아" 팀 최종 코드 파일입니다. 감사합니다!

### 1. 사용한 피쳐 
  - [period 1~ 10 새순] : 새순 9일의 평균에 대한 컬럼
  - [새순 max], [새순 min] 컬럼
  - [새순 diff] 컬럼= 새순 max - 새순 min
  - [나무 부피] 컬럼 = 수고 x 수관폭평균

### 2. 사용한 모델
  - RandomForest 그리드 서치
  - Xgboost 그리드 서치
  - 두 모델의 예측값 평균으로 최종 예측 결과 도출


#1. Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 0s (36.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

#2. 파생변수 생성

### NMAE 평가산식 코드

In [4]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

### 파생변수 생성
- period_i_새순 : 9일의 새순 평균 컬럼
- 새순 diff : 새순 max 컬럼 - 새순 min 컬럼
- 새순 max 컬럼
- 새순 min 컬럼
- 나무 부피 컬럼 : 수고(m) x 수관폭평균
- 엽록소 컬럼의 경우 target 값과의 연관성이 부족하다 생각해 all drop

In [5]:
def reduce_col3(df,trainortest):
    sprout=df.iloc[:,-178:-89] # 새순
    chlor=df.iloc[:,-89:] # 엽록소(chlorophyll)

    # 9일씩 묶어서 평균
    sarr=np.array(sprout.columns)
    sarr=np.append(sarr,'d').reshape(10,9)
    sdate_dic={
        'period_1 새순':sarr[0],
        'period_2 새순':sarr[1],
        'period_3 새순':sarr[2],
        'period_4 새순':sarr[3],
        'period_5 새순':sarr[4],
        'period_6 새순':sarr[5],
        'period_7 새순':sarr[6],
        'period_8 새순':sarr[7],
        'period_9 새순':sarr[8],
        'period_10 새순':sarr[9][:-1]
    }

    carr=np.array(chlor.columns)
    carr=np.append(carr,'d').reshape(10,9)
    cdate_dic={
        'period_1 엽록소':carr[0],
        'period_2 엽록소':carr[1],
        'period_3 엽록소':carr[2],
        'period_4 엽록소':carr[3],
        'period_5 엽록소':carr[4],
        'period_6 엽록소':carr[5],
        'period_7 엽록소':carr[6],
        'period_8 엽록소':carr[7],
        'period_9 엽록소':carr[8],
        'period_10 엽록소':carr[9][:-1]
    }

    new=pd.DataFrame()
    for period,dates in sdate_dic.items():
        new[period]=sprout[dates].mean(axis=1)

    #for period,dates in cdate_dic.items():
    #    new[period]=chlor[dates].mean(axis=1)
    ## 엽록소 걍 버림

    new['새순diff']=df['2022-09-01 새순']-df['2022-11-28 새순']
    new['새순max']=df['2022-09-01 새순']
    new['새순min']=df['2022-11-28 새순']

    # 다른 칼럼들도 예쁘게
    ## ex) 착과량(int) -> 착과량

    if trainortest=='train' :
        ## train은 착과량 포함
        new['착과량']=df['착과량(int)']

    new['나무부피']=df['수고(m)']*df['수관폭평균']
    return new

#3. 데이터 불러오기

In [7]:
#기본 경로 지정
%cd /content/drive/MyDrive/2022_상추의 생육 환경 생성 AI 경진대회/2022감귤

/content/drive/.shortcut-targets-by-id/1-EzLt_gX0CI4hJP3C7mM2Fn5AZVbAmPI/2022_상추의 생육 환경 생성 AI 경진대회/2022감귤


In [8]:
#train, test 셋 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [11]:
# 앞에서 만든 함수 적용 및 X, y split
X=reduce_col3(train,'train').drop('착과량',axis=1)
y=reduce_col3(train,'train')['착과량']

In [12]:
X.columns

Index(['period_1 새순', 'period_2 새순', 'period_3 새순', 'period_4 새순',
       'period_5 새순', 'period_6 새순', 'period_7 새순', 'period_8 새순',
       'period_9 새순', 'period_10 새순', '새순diff', '새순max', '새순min', '나무부피'],
      dtype='object')

#4-1. 모델링 - RandomForestRegressor()

## 그리드 서치

In [13]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import cross_val_score, cross_validate
import time

#시간 체크용
start = time.time()     

rf = RandomForestRegressor(n_jobs = -1, random_state = 0)
param_grid = {
        'max_depth':  [3, 4, 5, 6, 7],
        'bootstrap': [True, False],
        'max_features': ['auto', 'sqrt', 'log2'],
        'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
    }

ss=ShuffleSplit(test_size=0.3,random_state=0)
grid_rf_cv=GridSearchCV(rf,
                     return_train_score=True,
                       param_grid=param_grid,
                       cv=ss,
                       verbose=0,
                       scoring='neg_mean_absolute_error',
                       n_jobs=-1)

grid_rf_cv.fit(X,y)

end=time.time()
print('수행 시간: {0:.3f}'.format(end - start))
print('최적의 매개변수 조합: ', grid_rf_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_rf_cv.best_score_)

수행 시간: 1361.428
최적의 매개변수 조합:  {'bootstrap': False, 'criterion': 'friedman_mse', 'max_depth': 6, 'max_features': 'sqrt'}
최고의 교차 검증 점수:  -30.18017600138905


## 최적모델을 best_rf 에 저장 후 NMAE 확인

In [14]:
best_rf = grid_rf_cv.best_estimator_
y_pred_rf = best_rf.predict(X)
print(f'NMAE : {NMAE(y,y_pred_rf)}')

NMAE : 0.06610512938333356


## test data로 예측

In [15]:
X_test=reduce_col3(test,'test')
test_pred_rf = best_rf.predict(X_test)

In [23]:
test_pred_rf

array([247.50967039, 753.26316059, 151.29249931, ..., 365.18173387,
       238.79729648,  95.82418773])

# 4-2. 모델링 - XGboostRegressor()

In [16]:
import xgboost as xgb

## 그리드 서치

In [17]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV

xgbr=xgb.XGBRegressor()
param_grid = {
        'learning_rate':[0.001,0.01,0.1,0.2],
        'max_depth':  [3, 4, 5, 6, 7]
    }

ss=ShuffleSplit(test_size=0.3,random_state=0)
grid_xgb_cv=GridSearchCV(xgbr,return_train_score=True,
                       param_grid=param_grid,
                       cv=ss,
                       verbose=0,
                       scoring='neg_mean_absolute_error',
                       n_jobs=-1)

grid_xgb_cv.fit(X,y)

print('최적의 매개변수 조합: ', grid_xgb_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_xgb_cv.best_score_)

[04:11:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
최적의 매개변수 조합:  {'learning_rate': 0.1, 'max_depth': 3}
최고의 교차 검증 점수:  -30.6408947532533


## 최적모델을 best_xgb 에 저장 후 NMAE 확인

In [19]:
best_xgb=grid_xgb_cv.best_estimator_
y_pred_xgb=best_xgb.predict(X)
print(f'NMAE : {NMAE(y,y_pred_xgb)}')

NMAE : 0.06664120807503442


## test data로 예측

In [24]:
test_pred_xgb = best_xgb.predict(X_test)

# 5. 모델 앙상블

In [25]:
final_pred = 0.5*test_pred_rf + 0.5*test_pred_xgb

In [26]:
final_pred

array([247.88438232, 750.1590156 , 147.53183163, ..., 367.06430138,
       233.88457964,  96.13522695])

#6. 추출

In [27]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['착과량(int)'] = final_pred
sample_submission.to_csv('submit_final.csv', index=False)
sample_submission

,ID,착과량(int)
0,TEST_0000,247.884382
1,TEST_0001,750.159016
2,TEST_0002,147.531832
3,TEST_0003,440.817608
4,TEST_0004,691.450829
...,...,...
2203,TEST_2203,754.127800
2204,TEST_2204,338.579131
2205,TEST_2205,367.064301
2206,TEST_2206,233.884580


# 7. [추가] 교차검증 및 변수 중요도 확인

## 랜덤포레스트 교차검증

In [28]:
from sklearn.model_selection import cross_validate
cv_scores = cross_validate(rf,  X, y,                     #모델과 훈련 세트
                           cv = 5,                                  #5-fold
                           return_train_score = True,               #훈련 세트 점수도 반환
                           n_jobs = -1)

cv_scores_2 = cross_validate(best_rf, X, y,                    #모델과 훈련 세트
                             cv = 5,                                  #5-fold
                             return_train_score = True,               #훈련 세트 점수도 반환
                             n_jobs = -1)

print('첫 번째 교차 검증 결과')
print('훈련 세트 점수: {0:.3f}'.format(np.mean(cv_scores['train_score'])))
print('검증 세트 점수: {0:.3f}'.format(np.mean(cv_scores['test_score'])))
print('\n')
print('두 번째 교차 검증 결과')
print('훈련 세트 점수: {0:.3f}'.format(np.mean(cv_scores_2['train_score'])))
print('검증 세트 점수: {0:.3f}'.format(np.mean(cv_scores_2['test_score'])))

첫 번째 교차 검증 결과
훈련 세트 점수: 0.995
검증 세트 점수: 0.967


두 번째 교차 검증 결과
훈련 세트 점수: 0.979
검증 세트 점수: 0.970


## Xgboost 교차검증

In [29]:
from sklearn.model_selection import cross_validate
cv_scores = cross_validate(xgbr,  X, y,                     #모델과 훈련 세트
                           cv = 5,                                  #5-fold
                           return_train_score = True,               #훈련 세트 점수도 반환
                           n_jobs = -1)

cv_scores_2 = cross_validate(best_xgb, X, y,                    #모델과 훈련 세트
                             cv = 5,                                  #5-fold
                             return_train_score = True,               #훈련 세트 점수도 반환
                             n_jobs = -1)

print('첫 번째 교차 검증 결과')
print('훈련 세트 점수: {0:.3f}'.format(np.mean(cv_scores['train_score'])))
print('검증 세트 점수: {0:.3f}'.format(np.mean(cv_scores['test_score'])))
print('\n')
print('두 번째 교차 검증 결과')
print('훈련 세트 점수: {0:.3f}'.format(np.mean(cv_scores_2['train_score'])))
print('검증 세트 점수: {0:.3f}'.format(np.mean(cv_scores_2['test_score'])))

첫 번째 교차 검증 결과
훈련 세트 점수: 0.978
검증 세트 점수: 0.970


두 번째 교차 검증 결과
훈련 세트 점수: 0.978
검증 세트 점수: 0.970


## 랜덤포레스트 특성 중요도

In [30]:
#특성 중요도
importances = best_rf.feature_importances_

#특성 이름
features = X.columns

#데이터 프레임 생성
new = pd.DataFrame({'특성' : np.array(features), '중요도' : importances})

#가장 중요한 특성 5개
new.sort_values(by='중요도', ascending=False)

,특성,중요도
1,period_2 새순,0.206323
2,period_3 새순,0.203110
0,period_1 새순,0.155636
11,새순max,0.147918
3,period_4 새순,0.094341
4,period_5 새순,0.062433
10,새순diff,0.045945
6,period_7 새순,0.034997
5,period_6 새순,0.031448
7,period_8 새순,0.009795


## XGboost 특성 중요도

In [31]:
#특성 중요도
importances = best_xgb.feature_importances_

#특성 이름
features = X.columns

#데이터 프레임 생성
new = pd.DataFrame({'특성' : np.array(features), '중요도' : importances})

#가장 중요한 특성 5개
new.sort_values(by='중요도', ascending=False)

,특성,중요도
0,period_1 새순,0.558243
1,period_2 새순,0.244044
11,새순max,0.082362
2,period_3 새순,0.047165
10,새순diff,0.043126
3,period_4 새순,0.010819
12,새순min,0.007358
4,period_5 새순,0.002545
6,period_7 새순,0.000885
9,period_10 새순,0.000765
